# Code to train model on recognizing different hand signs.

In [1]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
#load data
data_dict = pickle.load(open('data.p', 'rb'))

data = data_dict['data']
labels = data_dict['labels']

#stratify is used to maintain propotin between the different gestures during training and testing
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.3, shuffle = True, stratify = labels)

In [4]:
print(len(y_test))

1412


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input = nn.Linear(2*21, 38)
        self.hidden = nn.Linear(38, 38)
        self.output = nn.Linear(38, 26)

    def forward(self, x):
        x = x.view(-1, 2*21)
        x = F.relu(self.input(x))
        x = F.relu(self.hidden(x))
        x = F.log_softmax(self.output(x), dim=1)
        return x
    
model = NeuralNetwork()

In [6]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (input): Linear(in_features=42, out_features=38, bias=True)
  (hidden): Linear(in_features=38, out_features=38, bias=True)
  (output): Linear(in_features=38, out_features=26, bias=True)
)


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 20
for epoch in range(epochs):
    for images, labels in zip(x_train, y_train):
        images = torch.tensor(images).to(device)
        images = torch.flatten(images).unsqueeze(0)  # Add batch dimension if needed

        labels = torch.tensor(int(labels)).to(device)
        
        optimizer.zero_grad()
        output = model(images)
        loss = loss_function(output, torch.tensor(int(labels)))
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')


ValueError: Expected input batch_size (1) to match target batch_size (0).

In [ ]:
# Train the model with training data
model = RandomForestClassifier()
model.fit(x_train, y_train)

In [ ]:
# Test model
y_predict = model.predict(x_test)
score = accuracy_score(y_predict, y_test)
print('{}% of samples were classified correctly !'.format(score * 100))

In [ ]:
# Save model for model inference
f = open('model.p', 'wb')
pickle.dump([model], f)
f.close()